In [12]:
TYPE_PC = False # True For Mac   -   False Ubuntu

In [13]:
### Import libraries

import sys

if TYPE_PC:
    sys.path.append('/home/daniel/Documents/phd/phd-thesis-lab/12-third_year/05-CommunityDetection/')
else:
    sys.path.append('/home/daniel/Documents/phd/phd-thesis-lab/12-third_year/05-CommunityDetection/')

import pandas as pd
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
import auxiliar_path

In [20]:
### Global variables
### Global variables

DATASET = "AMZ" # AMZ, HC, PM, UN, TOY
NODE_TYPE = True

PATH_DATASET = auxiliar_path.get_path_dataset(DATASET)
PATH_NODETYPE = auxiliar_path.get_path_topbot(NODE_TYPE)

if TYPE_PC:
    GLOBAL_PATH = "/Users/ddiaz/Documents/code/phd-thesis-lab/12-third_year/"    
else:
    GLOBAL_PATH = "/home/daniel/Documents/phd/phd-thesis-lab/12-third_year/"

# File CSVpa
### GLOBAL variables
GRAPH_PATH = GLOBAL_PATH + "00-Data/" + PATH_DATASET + "/02-Graphs/"+PATH_NODETYPE+"/AMZ_top_hyperbolic_DF_alpha1.graphml"
print(GRAPH_PATH)

/home/daniel/Documents/phd/phd-thesis-lab/12-third_year/00-Data/01-AMZ/02-Graphs/01-Top/AMZ_top_hyperbolic_DF_alpha1.graphml


In [131]:
### Read GRAPH

g = ig.read(GRAPH_PATH)
print(g.summary(), "\n")

print("Graph info:")
print("\t|V| =", g.vcount())
print("\t|E| =", g.ecount())
print("\t d  =", g.density()) 

IGRAPH U-WT 9560 90843 -- 
+ attr: freq (v), id (v), type (v), weight (e) 

Graph info:
	|V| = 9560
	|E| = 90843
	 d  = 0.001988158982684504


In [138]:
### Community Detection

comms = g.community_multilevel(weights=g.es["weight"])
g.vs["cls"] = comms.membership
print(comms.summary(), "\n")

Clustering with 9560 elements and 1405 clusters 



X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X


In [139]:
len(set(g.vs["cls"]))

1405

In [140]:
comms_with_one = {subgraph.vs["cls"][0]: subgraph for i, subgraph in enumerate(comms.subgraphs()) if len(subgraph.vs) > 5}
comms_to_remove = [sg for sg in comms.subgraphs() if len(sg.vs) <= 5]
comms_with_one, len(comms_to_remove)

({0: <igraph.Graph at 0x7820d03d4310>,
  1: <igraph.Graph at 0x7820d03d4400>,
  2: <igraph.Graph at 0x7820d03d44f0>,
  3: <igraph.Graph at 0x7820d03d45e0>,
  4: <igraph.Graph at 0x7820d03d46d0>,
  5: <igraph.Graph at 0x7820d03d47c0>,
  6: <igraph.Graph at 0x7820d03d48b0>,
  7: <igraph.Graph at 0x7820d03d49a0>,
  8: <igraph.Graph at 0x7820d03d4a90>,
  9: <igraph.Graph at 0x7820d03d4b80>,
  10: <igraph.Graph at 0x7820d03d4c70>,
  11: <igraph.Graph at 0x7820d03d4e50>,
  12: <igraph.Graph at 0x7820d03d4f40>,
  13: <igraph.Graph at 0x7820d03d5030>,
  15: <igraph.Graph at 0x7820d03d5210>,
  16: <igraph.Graph at 0x7820d03d5300>,
  17: <igraph.Graph at 0x7820d03d53f0>,
  19: <igraph.Graph at 0x7820d03d55d0>,
  21: <igraph.Graph at 0x7820d03d57b0>,
  23: <igraph.Graph at 0x7820d03d5990>,
  34: <igraph.Graph at 0x7820d03d63e0>,
  37: <igraph.Graph at 0x7820d03d66b0>,
  60: <igraph.Graph at 0x7820d03d7c40>,
  80: <igraph.Graph at 0x7820d04a8f40>,
  86: <igraph.Graph at 0x7820d04a94e0>,
  97: <igr

In [141]:
nodes_to_remove = []
for i in comms_to_remove:
    for node in i.vs:
        nodes_to_remove.append(node["id"])
len(nodes_to_remove)

1455

In [143]:
asd = []
for node in nodes_to_remove:
    asd.append(g.vs.find(id_eq=node).index)

In [144]:
g.delete_vertices(asd)

In [145]:
g.summary()

'IGRAPH U-WT 8105 90632 -- \n+ attr: cls (v), freq (v), id (v), type (v), weight (e)'

In [149]:
new_ids_comms = 1500
for com in comms_with_one.values():
    if com.density() > 0.1:
        print("X")
        new_partition = com.community_multilevel(weights=com.es["weight"])
        for new_comms in new_partition.subgraphs():
            for nodes in new_comms.vs():
                node_to_add = g.vs.find(id_eq=nodes["id"])
                node_to_add["cls"] = new_ids_comms
            new_ids_comms += 1

X
X
X
X
X
X
X


In [155]:
len(set(g.vs["cls"]))

37

In [156]:
g.write_graphml("g_comms_2.graphml")

In [37]:
test_node = g.vs[1010]

In [38]:
internal_external_degree_node(test_node)

(6, 1)

In [128]:
def internal_external_degree_node(node):
    """Compute the internal degree k_i^int of node i in a community C."""
    node_neighs = node.neighbors()
    node_community = node["cls"]

    internal_degree = [1 for i in node_neighs if i["cls"]==node_community]
    external_degree = [1 for i in node_neighs if i["cls"]!=node_community]
    
    return sum(internal_degree), sum(external_degree)


def check_strong_community(graph, communities):
    """Returns if it is a strong community."""

    dict_to_ret = {}

    for (id_c, commm) in communities.items():
        flag_weak_comm = False
        porcentaje = 0
        for node in commm.vs:
            vertex = g.vs.find(id_eq=node["id"])
            int_degree, ext_degree = internal_external_degree_node(vertex)
            if int_degree <= ext_degree:
                flag_weak_comm = True
                porcentaje += 1
        
        if flag_weak_comm:
            dict_to_ret[id_c] = [commm, False, porcentaje/commm.vcount()]
        else:
            dict_to_ret[id_c] = [commm, True, 0]

    return dict_to_ret

In [121]:
test = g.vs[0]
internal_external_degree_node(test)

(10, 8)

In [129]:
check_strong_community(g, comms_with_one)

{0: [<igraph.Graph at 0x7820d34462f0>, False, 0.36075949367088606],
 1: [<igraph.Graph at 0x7820d34463e0>, False, 0.2033271719038817],
 2: [<igraph.Graph at 0x7820d34466b0>, False, 0.30141843971631205],
 3: [<igraph.Graph at 0x7820d34467a0>, False, 0.09051724137931035],
 4: [<igraph.Graph at 0x7820d3446980>, False, 0.16901408450704225],
 5: [<igraph.Graph at 0x7820d3446a70>, False, 0.11479028697571744],
 6: [<igraph.Graph at 0x7820d34457b0>, False, 0.23125],
 7: [<igraph.Graph at 0x7820d3445f30>, False, 0.3470319634703196],
 8: [<igraph.Graph at 0x7820d3445030>, False, 0.07983761840324763],
 9: [<igraph.Graph at 0x7820d3446b60>, False, 0.3547400611620795],
 10: [<igraph.Graph at 0x7820d3446c50>, False, 0.24890829694323144],
 11: [<igraph.Graph at 0x7820d3446d40>, False, 0.1111111111111111],
 12: [<igraph.Graph at 0x7820d3446e30>, False, 0.3058103975535168],
 14: [<igraph.Graph at 0x7820d3447010>, False, 0.32592592592592595],
 15: [<igraph.Graph at 0x7820d3447100>, True, 0],
 17: [<igra